In [ ]:
import numpy as np
import pandas as pd
import cv2
from glob import glob

In [ ]:
images_path='C:\\Users\\Rishav\\Image caption\\Images\\' 
images=glob(images_path + '*.jpg')#reading all images ending with jpg
#len(images)

In [ ]:
#images[:5]

In [ ]:
import matplotlib.pyplot as plt
for i in range(6):
    plt.figure()
    img=cv2.imread(images[i])
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)#showing the images by converting in to RGB first
   # plt.imshow(img)

In [ ]:
from keras.applications import ResNet50 #downloding ResNet50
incept_model=ResNet50(include_top=True)

In [ ]:
#incept_model.summary() #summary of ResNet50 model

In [ ]:
from keras.models import Model
last=incept_model.layers[-2].output
modele=Model(inputs=incept_model.inputs,outputs=last)#removing the last layer in the model
#modele.summary()

In [ ]:
images_features={} #data preprocessing Starts
count=0
for i in images:
    img=cv2.imread(i)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)#conerting the given images to RGB
    img=cv2.resize(img,(224,224))
    
    img=img.reshape(1,224,224,3)
    pred=modele.predict(img).reshape(2048)#returning the values as 2048 vector
     
    img_name=i.split('\\')[-1] #slicing the image name from path
    images_features[img_name]=pred#adding the image feature to the dictionary
    
    count=count+1
    
    if count>8000:
        break
    elif count % 50 == 0:
        print(count)

In [ ]:
caption_path = 'C:\\Users\\Rishav\\Image caption\\captions.txt'#text preprocessing starts
captions = open(caption_path, 'rb').read().decode('utf-8').split('\n')#splitting the captions
len(captions)

In [ ]:
#captions

In [ ]:
captions_dict = {}
for i in captions:
    try:
        img_name = i.split(',')[0]
        caption = i.split(',')[1]
        if img_name not in captions_dict:
            captions_dict[img_name] = [caption] #assigning keys and values to images
                
        else:
            captions_dict[img_name].append(caption)
            
    except:
        pass

In [ ]:
captions_dict.pop('Caption','Image')

In [ ]:
captions_dict

In [ ]:
def lowers(st):
    modified=st.lower()
    modified="start "+modified+" end" #lower case all the values
    return modified

In [ ]:
for k,v in captions_dict.items():
    for vv in v:
        captions_dict[k][v.index(vv)]=lowers(vv)#looping through all captions

In [ ]:
#captions_dict

In [ ]:
count_words={}
count=1
for k,vv in captions_dict.items():
    for v in vv:
        for word  in v.split():
            if word not in count_words:
                count_words[word]=count 
                count=count+1
        
            

In [ ]:
#(count_words)

In [ ]:
for k,vv in captions_dict.items():
    for v in vv:
        encoded=[]
        for word in v.split():
            encoded.append(count_words[word]) #encoding the words
            
            
        captions_dict[k][vv.index(v)]=encoded
        

In [ ]:
#captions_dict

In [ ]:
MAX_LEN=0
for k,vv in captions_dict.items():
    for v in vv:
        if len(v) > MAX_LEN:
            MAX_LEN=len(v)#size of encoded sentences 
MAX_LEN     

In [ ]:
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

VOCAB_SIZE = len(count_words)

def generator(photo, caption):
    n_samples = 0
    
    X = []#image features here
    y_in = []
    y_out = []
    a=0
    for k, vv in caption.items():
        for v in vv:
            for i in range(1, len(v)):
                X.append(photo[k])

                in_seq= [v[:i]]
                out_seq = v[i]

                in_seq = pad_sequences(in_seq, maxlen=MAX_LEN, padding='post', truncating='post')[0]
                out_seq = to_categorical([out_seq], num_classes=VOCAB_SIZE)[0]

                y_in.append(in_seq)
                y_out.append(out_seq)
        a=a+1
        if a==8000:
            break
    return X, y_in, y_out
    

In [ ]:
X, y_in, y_out = generator(images_features, captions_dict)

In [ ]:
images_features

In [ ]:
len(X), len(y_in), len(y_out)

In [ ]:
X = np.array(X)
y_in = np.array(y_in, dtype='float64')
y_out = np.array(y_out, dtype='float64') 

In [ ]:
X.shape, y_in.shape, y_out.shape

In [ ]:
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import add
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model

In [ ]:
embedding_size = 128
max_len = MAX_LEN
vocab_size = len(count_words)

image_model = Sequential()

image_model.add(Dense(embedding_size, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_len))

image_model.summary()

language_model = Sequential()

language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))

language_model.summary()

conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(128, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model = Model(inputs=[image_model.input, language_model.input], outputs = out)


model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model,show_shapes=True)

In [ ]:
model.fit([X, y_in], y_out, batch_size=512, epochs=50)

In [ ]:
model.save_weights("C:\\Users\\Rishav\\Image caption")